### Importando nossas bibliotecas

In [1]:
import sqlite3
import pandas as pd

### Criando conexão com nosso DB

In [2]:
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

### Criando nossas tabelas (Clientes e Pedidos)

In [3]:
cursor.execute('''
               CREATE TABLE IF NOT EXISTS clientes (
                  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
                  nome TEXT,
                  email TEXT
               )
               '''
)

cursor.execute('''
               CREATE TABLE IF NOT EXISTS pedidos (
                  id INTEGER PRIMARY KEY,
                  produto TEXT,
                  valor REAL,
                  cliente_id INTEGER,
                  FOREIGN KEY(cliente_id) REFERENCES clientes(id)
               )
               '''
)


### Inserindo dados em nossa tabelas

In [4]:
cursor.execute("INSERT INTO clientes (nome, email) VALUES (?, ?)", ('otavio', 'otavio@gmail.com'))
cursor.execute("INSERT INTO clientes (nome, email) VALUES (?, ?)", ('danilo', 'danilo@hotmail.com'))

produto = input('Qual o nome do produto?')
valor = float(input('Qual o valor do produto?'))
cliente_id = int(input('Qual o id do cliente que comprou?'))

cursor.execute("INSERT INTO pedidos (produto, valor, cliente_id) VALUES (?, ?, ?)", (produto, valor, cliente_id))

conn.commit()


In [5]:
produto = input('Qual o nome do produto?')
valor = float(input('Qual o valor do produto?'))
cliente_id = int(input('Qual o id do cliente que comprou?'))

cursor.execute("INSERT INTO pedidos (produto, valor, cliente_id) VALUES (?, ?, ?)", (produto, valor, cliente_id))


conn.commit()

### Query que retorna pedidos de um determinado cliente utilizando LIKE

In [12]:
cursor.execute('''
               SELECT pedidos.produto, pedidos.valor
               FROM pedidos
               JOIN clientes ON pedidos.cliente_id = clientes.id
               WHERE clientes.nome LIKE 'João'
               '''
)
            
resultados = cursor.fetchall()
for resultado in resultados:
    print(resultado)

('Camiseta', 50.0)
('Tênis', 200.0)
('Camiseta', 50.0)
('Tênis', 200.0)
('Camiseta', 50.0)
('Tênis', 200.0)


In [14]:
resultados

[('Camiseta', 50.0),
 ('Tênis', 200.0),
 ('Camiseta', 50.0),
 ('Tênis', 200.0),
 ('Camiseta', 50.0),
 ('Tênis', 200.0)]

### Query que conta quantidade de pedidos por cliente utilizando COUNT

In [15]:
cursor.execute('''
               SELECT clientes.nome, COUNT(pedidos.id)
               FROM clientes
               JOIN pedidos ON clientes.id = pedidos.cliente_id
               GROUP BY clientes.id
               '''
)

resultados = cursor.fetchall()
for resultado in resultados:
    print(resultado)


('João', 6)
('Maria', 3)
('otavio', 1)


### Query que retorna o valor total gasto por cliente utilizando SUM

In [16]:
cursor.execute('''
               SELECT clientes.nome, SUM(pedidos.valor)
               FROM clientes
               JOIN pedidos ON clientes.id = pedidos.cliente_id
               GROUP BY clientes.id
               '''
)

resultados = cursor.fetchall()
for resultado in resultados:
    print(resultado)


('João', 750.0)
('Maria', 300.0)
('otavio', 120.0)


## Conteúdo Bonus

### Transformando nossos dados em Pandas.DataFrame

In [17]:
cursor.execute('''
               SELECT * 
               FROM pedidos
               '''
)

rows_pedidos = cursor.fetchall()

df_pedidos = pd.DataFrame(rows_pedidos, columns=[column[0] for column in cursor.description])

df_pedidos.head()

,id,produto,valor,cliente_id
0,1,Camiseta,50.0,1
1,2,Tênis,200.0,1
2,3,Calça,100.0,2
3,4,Camiseta,50.0,1
4,5,Tênis,200.0,1


In [18]:
cursor.execute('''
               SELECT * 
               FROM clientes
               '''
)

rows_clientes = cursor.fetchall()

df_clientes = pd.DataFrame(rows_clientes, columns=[column[0] for column in cursor.description])

df_clientes.head()

,id,nome,email
0,1,João,joao@gmail.com
1,2,Maria,maria@hotmail.com
2,3,João,joao@gmail.com
3,4,Maria,maria@hotmail.com
4,5,otavio,otavio@gmail.com


### Unindo nossas tabelas com pandas

In [19]:
df_merged = df_clientes.merge(df_pedidos, left_on='id', right_on='cliente_id', how='left')

In [20]:
df_merged

,id_x,nome,email,id_y,produto,valor,cliente_id
0,1,João,joao@gmail.com,1.0,Camiseta,50.0,1.0
1,1,João,joao@gmail.com,2.0,Tênis,200.0,1.0
2,1,João,joao@gmail.com,4.0,Camiseta,50.0,1.0
3,1,João,joao@gmail.com,5.0,Tênis,200.0,1.0
4,1,João,joao@gmail.com,7.0,Camiseta,50.0,1.0
5,1,João,joao@gmail.com,8.0,Tênis,200.0,1.0
6,2,Maria,maria@hotmail.com,3.0,Calça,100.0,2.0
7,2,Maria,maria@hotmail.com,6.0,Calça,100.0,2.0
8,2,Maria,maria@hotmail.com,9.0,Calça,100.0,2.0
9,3,João,joao@gmail.com,NaN,NaN,NaN,NaN


### Criando um Dataframe somente com Pedidos do João

In [21]:
df_joao = df_merged.loc[df_merged['nome'] == 'João', ['nome', 'produto', 'valor']]

In [22]:
df_joao

,nome,produto,valor
0,João,Camiseta,50.0
1,João,Tênis,200.0
2,João,Camiseta,50.0
3,João,Tênis,200.0
4,João,Camiseta,50.0
5,João,Tênis,200.0
9,João,NaN,NaN


### Obtendo valor total de compras do João

In [23]:
print(f'Valor total de compras do João:', df_joao['valor'].sum())

Valor total de compras do João: 750.0
